<center><h1>À propos du revenu universel</h1><i>le 29 mai 2016</i></center>

In [1]:
from IPython.display import HTML
import base64
def show_image(filename, width="90%"):
    img = open(filename,'rb').read()
    html = '<center><img src="data:text/html;base64,{}" style="width:{};"></img></center>'.format(
        base64.b64encode(img).decode('utf8'), width)
    return HTML(html)

HTML("""
    <style>
    body {
      font-size: 20px;
      font-family: 'Lora', serif;
      color: #444;
      }

    h1 {
      font-size: 180%;
      line-height: 135%;
      font-family: 'Open Sans', sans-serif;
      text-align: center;
      margin: 0 0 60px;
      padding: 0 40px;
      }

    h1, h2, h3, h4, h5, h6 { 
      padding: 0 40px; 
      margin-top: 40px;
      color: #444;
      }

    h2 {
        font-size: 28px;
        font-style: italic;
        font-family: 'Lora', serif;
    }

    p {
      font-family: 'Lora', serif;
      line-height: 160%;
      font-size: 100%;
      margin-bottom: 0;
      color: #444;
      }

    </style>

    <script>
    $('div.input').show();
    $('div.output_prompt').css('display','flex');
    $('div.input_prompt').css('display','flex');
    
    
    var filename = "http://fonts.googleapis.com/css?family=Open+Sans:400italic,400,300,700|Lora:400,700,400italic";
    var fileref=document.createElement("link")
        fileref.setAttribute("rel", "stylesheet")
        fileref.setAttribute("type", "text/css")
        fileref.setAttribute("href", filename)
    document.getElementsByTagName("head")[0].appendChild(fileref)
    </script>

""")

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.xkcd()
pass

Depuis quelques semaines, on entend se développer l'idée du *revenu universel* (ou *revenu de base inconditionnel*). Cela consiste à remplacer toutes les allocations et assurances issues de la solidarité nationale par une unique prime qui serait versée à chacun, sans aucune condition. Cette idée va faire l'objet d'[une votation fédérale en Suisse le 5 juin prochain](http://initiative-revenudebase.ch), et est également en débat en Finlande.

L'idée a déjà fait réagir beaucoup de monde qui y voient tantôt le comble de l'assistanat -- puisque la prime est versée sans condition d'activité -- ou la pire des injustice -- puisque les riches la recevrait au même titre que les pauvres.

Plutôt que de réagir le plan moral (où le débat est indispensable mais généralement assez long), j'ai voulu partager quelques réflexions techinques sur le sujet.


## Simplifier la *fonction fiscale*

La *fonction fiscale* est une fonction mathématique qui fait le lien entre le [revenu primaire]() (ce que je coûte à mon employeur) et le [revenu disponible](https://fr.wikipedia.org/wiki/Revenu_disponible_brut) (ce que je peux dépenser pour de vrai). *(Le terme "fonction fiscale" est de moi car je n'ai pas trouvé de nom officiel à cette fonction ; si quelqu'un en connait un ou a une meilleur suggestion, je serais ravi d'avoir son retour.)*

En France, on calcule la fonction fiscale en 4 étapes:

* **salaire brut = revenu primaire - cotisations patronales**. Les cotisations patronales sont progressives jusqu'à 12'344€ mensuels, puis dégressives au delà. En moyenne, elles correspondent environ à 30% du revenu primaire.

* **salaire net = salaire brut - cotisations salariales**. Les cotisations salariales sont progressives jusqu'à 
3'086€ mensuels, puis dégressives au delà. En moyenne, elles correspondent environ à 17.5% du revenu primaire (25% du salaire brut). Au final, le salaire net vaut donc 52% du revenu primaire. 

* **revenu apres impot = salaire net - impots**. Je ne rentre pas dans le détail ; il suffit d'avoir fait une déclaration d'impôt pour tout comprendre du mode de calcul ;-) Je rapelle simplement que les impôts que l'on paye ne dépendent pas seulement des revenus, mais de plusieurs autres paramètres: quel est votre logement (pour les taxes foncières et habitation) ? avez-vous du capital (pour l'ISF) ? oui mais sans les oeuvres d'art ? regardez-vous la télévision ? quand avez-vous acheté votre maison ? était-ce du neuf ou de l'ancien ? etc.

* **revenu disponible = revenu apres impots + prestations sociales**. Pareil, je renonce à entrer dans le détail puisqu'elles dépendent de votre âge, de votre santé, de votre famille, de votre situation professionelle, de votre logement et de votre situation financière. Rappelons simplement que certaines prestations sont des **assurances** (santé, chômage) tandis que d'autres relèvent de la solidarité ou de la redistribution.


J'ai un peu simplifié (les puristes me pardonneront-ils ?) puisque ce schéma vaut pour les salariés du privé ; ils existe des cas particuliers pour les fonctionnaires, les professions indépendantes, les chômeurs, les intermittants, les étudiants, les retraités, les étrangers, les français résidant à l'étranger...


Bref, vous comprenez que tracer la *fonction fiscale* est un exercice voué à l'échec. Moyennant beaucoup d'hypothèses et approximations elle ressemble à ceci:

In [3]:
pos = lambda x: 0.5*(x+np.abs(x))

import numpy as np

if False:
    smic=2343
    gmax = 6000
    x = np.arange(0,gmax)
    brut = x - 0.3742*x - pos(x-3086)*0.0808
    net = brut - 0.1370*x +pos(x-3086)*0.1045
    #y = (x>=1600*1.04)*(0.5*x) + (x<1600*1.04)*(514+0.62*x*0.5)
    #
    fig=plt.figure(figsize=(10,8))
    ax = plt.gca()
    ax.plot(x,x, 'k-')
    ax.plot(x,brut)
    ax.plot(x,net)
    #ax.plot(x,500+0.35*x)
    impot = pos(net-808)*0.14 + pos(net-2232)*0.16 + pos(net-5985)*0.11
    rsa = (514 + 0.62*net - net)*(x<1.04*smic)
    dispo = net + rsa - impot
    ax.plot(x,dispo, 'g')

    #ax.plot(x,y, 'r')
    ax.plot(smic+0*x, x, 'k:')
    ax.text(smic, -50, 'SMIC', ha='center', va='top')
    ax.set_ylim(0,gmax)
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['bottom'].set_color('none')
    plt.xticks([])
    plt.yticks([])

    plt.text(gmax*1.01, 0, 'revenu primaire', ha='left', va='center')
    plt.text(gmax*1.01, max(brut), 'salaire brut', color='b', ha='left', va='center')
    plt.text(gmax*1.01, max(x), 'revenu primaire', color='k', ha='left', va='center')
    plt.text(gmax*1.01, max(net), 'salaire net', color='r', ha='left', va='center')
    #plt.text(gmax*1.02, max(dispo), 'revenu primaire', color='k', ha='left', va='center')
    plt.text(gmax*1.01, max(dispo), 'revenu disponible', color='g', ha='left', va='center')
    ax.set_aspect('equal')
    plt.plot(x,0*x,'k')
    plt.plot(0*x,x,'k')
    ax.set_xlim(-100,8000)
    ax.set_ylim(-100,6000)
    pass
    fig.savefig('./img/2016-05-29-today.png', bbox_inches='tight')

show_image('./img/2016-05-29-today.png', width="600px")

Cette courbe (qui ne prend en compte que les cotisations sociales, le RSA et l'IR et qui ne prend pas en compte les revenus supérieurs à 6000€ mensuels) fait intervenir 14 paramètres.

On serait tenté de résumer par une formule plus simple:

* Chacun reçoit 500€ de *revenu universel*.
* Chaque euro est taxé à un taux fixe de 65%.

On trouve alors une *fonction fiscale simplifiée*, quasi-identique à l'actuelle (un peu plus généreuse de quelques milliards):

In [4]:
pos = lambda x: 0.5*(x+np.abs(x))

import numpy as np

if False:
    smic=2343
    gmax = 6000
    x = np.arange(0,gmax)
    brut = x - 0.3742*x - pos(x-3086)*0.0808
    net = brut - 0.1370*x +pos(x-3086)*0.1045
    #y = (x>=1600*1.04)*(0.5*x) + (x<1600*1.04)*(514+0.62*x*0.5)
    #
    fig=plt.figure(figsize=(10,8))
    ax = plt.gca()
    ax.plot(x,x, 'k-')
    #ax.plot(x,brut)
    #ax.plot(x,net)
    #ax.plot(x,500+0.35*x)
    impot = pos(net-808)*0.14 + pos(net-2232)*0.16 + pos(net-5985)*0.11
    rsa = (514 + 0.62*net - net)*(x<1.04*smic)
    dispo = net + rsa - impot
    ax.plot(x,dispo, 'g')
    ax.plot(x,500+0.35*x, 'c')

    #ax.plot(x,y, 'r')
    ax.plot(smic+0*x, x, 'k:')
    ax.text(smic, -50, 'SMIC', ha='center', va='top')
    ax.set_ylim(0,gmax)
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['bottom'].set_color('none')
    plt.xticks([])
    plt.yticks([])

    plt.text(gmax*1.01, 0, 'revenu primaire', ha='left', va='center')
    #plt.text(gmax*1.01, max(brut), 'salaire brut', color='b', ha='left', va='center')
    plt.text(gmax*1.01, max(x), 'revenu primaire', color='k', ha='left', va='center')
    #plt.text(gmax*1.01, max(net), 'salaire net', color='r', ha='left', va='center')
    #plt.text(gmax*1.02, max(dispo), 'revenu primaire', color='k', ha='left', va='center')
    plt.text(gmax*1.01, max(dispo), 'revenu disponible actuel', color='g', ha='left', va='top')
    plt.text(gmax*1.01, max(dispo)*1.05, 'fonction fiscale simplifiée', color='c', ha='left', va='bottom')
    ax.set_aspect('equal')
    plt.plot(x,0*x,'k')
    plt.plot(0*x,x,'k')
    ax.set_xlim(-100,8000)
    ax.set_ylim(-100,6000)
    pass
    fig.savefig('./img/2016-05-29-rbi.png', bbox_inches='tight')

show_image('./img/2016-05-29-rbi.png', width="600px")

Bien sûr, on peut moduler ces deux paramètres pour être d'avantage *pro-riche* ou *pro-pauvres*. On peut aussi faire baisser le taux (incroyable et inassumable) de 65% en privatisant certaine prestations sociales.

## Supprimer le SMIC

Il y a deux manières de considérer le SMIC. On peut classiquement dire que le SMIC interdit à un patron de trop mal payer un employé. Mais on peut aussi y voir une interdiction envers les salariés peu qualifiés: ceux dont la productivité est inférieure à 15.45 euros par heure n'ont pas le droit à un emploi salarié.

Dès lors, les courbes tracées au paragraphe précédent ne sont que théoriques. Si personne ne peut (à temps complet) être payé moins que le SMIC, la courbe devient en pratique:

In [5]:
pos = lambda x: 0.5*(x+np.abs(x))

import numpy as np

if False:
    smic=2343
    gmax = 6000
    x = np.arange(0,gmax)
    brut = x - 0.3742*x - pos(x-3086)*0.0808
    net = brut - 0.1370*x +pos(x-3086)*0.1045
    #y = (x>=1600*1.04)*(0.5*x) + (x<1600*1.04)*(514+0.62*x*0.5)
    #
    fig=plt.figure(figsize=(10,8))
    ax = plt.gca()
    #ax.plot(x,brut)
    #ax.plot(x,net)
    #ax.plot(x,500+0.35*x)
    impot = pos(net-808)*0.14 + pos(net-2232)*0.16 + pos(net-5985)*0.11
    rsa = (514 + 0.62*net - net)*(x<1.04*smic)
    dispo = net + rsa - impot
    dispo = dispo*(x>smic) + 514*(x<=smic)
    ax.plot(x,dispo, 'g')
    #ax.plot(x,500+0.35*x, 'c')

    #ax.plot(x,y, 'r')
    ax.plot(smic+0*x, x, 'k:')
    ax.text(smic, -50, 'SMIC', ha='center', va='top')
    ax.set_ylim(0,gmax)
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['bottom'].set_color('none')
    plt.xticks([])
    plt.yticks([])

    plt.text(gmax*1.01, 0, 'capacité de production', ha='left', va='center')
    #plt.text(gmax*1.01, max(brut), 'salaire brut', color='b', ha='left', va='center')
    plt.text(gmax*1.01, max(x), 'revenu primaire', color='r', ha='left', va='center')
    #plt.text(gmax*1.01, max(net), 'salaire net', color='r', ha='left', va='center')
    #plt.text(gmax*1.02, max(dispo), 'revenu primaire', color='k', ha='left', va='center')
    plt.text(gmax*1.01, max(dispo), 'revenu disponible', color='g', ha='left', va='top')
    ax.set_aspect('equal')
    plt.plot(x,0*x,'k')
    plt.plot(0*x,x,'k')
    ax.plot(x,x*(x>smic), 'r-')
    ax.set_xlim(-100,8000)
    ax.set_ylim(-100,6000)
    pass
    fig.savefig('./img/2016-05-29-allowed.png', bbox_inches='tight')

show_image('./img/2016-05-29-allowed.png', width="600px")

La partie à gauche de la courbe correpond aux chômeurs qui subissent de plein fouet le plus gros effet de seuil de notre système.

Bien sûr je suis conscient des abus pouvant advenir si le SMIC était supprimé ; mais la combinaison d'un revenu universel et d'une flat tax (comme décrit plus haut) permet:

* De donner à chacun un revenu décent, même s'il est la combinaison d'un salaire et d'une aide.
* De laisser toute la population participer à l'effort de production, à hauteur de ses moyens.
* De ne laisser personne sans emploi ; avec le risque de *déconnection sociale* qui en découle.

Pour limiter les abus, on doit pouvoir imaginer une contrainte (ou une incitation) plus intelligente que celle du SMIC ; en utilisant par exemple le [coefficient de Gini](https://fr.wikipedia.org/wiki/Coefficient_de_Gini) qui mesure l'uniformité de répartition de la masse salariale.

## Vers un impôt négatif progressif

Finalement, le revenu universel peut être vu comme le revenu minimum que l'on puisse obtenir ;
qui correspond généralement au revenu disponible pour un revenu primaire nul.
De ce point de vue, on peut pratiquement affirmer qu'il existe aujourd'hui un revenu universel de 514€ (le RSA)
auquel s'ajoute les revenus du travail et leur fiscalité.

Ce qui me semble intéressant, c'est de considérer la fonction fiscale dans son ensemble, et de définir ses propriétés souhaitables. Voici quelques exemples:

* **On peut souhaiter qu'elle commence à zéro**: c'est le principe énoncé par les détracteurs du revenu universel ;
il faut exercer une activité (même minime) pour bénéficier de la solidarité nationale.

* **On peut souhaiter (à l'inverse) qu'elle ait un plancher**: il semble évident que personne ne doit avoir à payer plus
qu'il ne gagne (autrement dit `ff(x)>0`), mais le revenu universel consiste justement à garantir que ce plancher vaut
plusieurs centaines d'euros.

* **On peut souhaiter qu'elle ait un plafond** (une idée pour l'extrême gauche): personne ne peut gagner plus qu'un certain montant.

* **On peut souhaiter qu'elle soit constante**. C'est le parfait (?!) communisme: tout le monde gagne la même chose.

* **On peut souhaiter qu'elle ne dépasse pas un certain taux d'imposition**: je crois qu'on considère
aujourd'hui comme *confiscatoire* une fonction fiscale qui ne garantit pas `ff(x)>0.3 x` (c'est à dire qu'on ne payer pas plus de 70% d'impôt).

* **On peut souhaiter qu'elle soit strictement croissante**: Quand je fais un effort pour augmenter mon revenu primaire,
je voudrais voir croître mon revenu disponible.

* **On peut souhaiter qu'elle soit continue** pour éviter les *effets de seuils*: un centime de plus ou de moins ne doit
pas affecter le revenu disponible de plusieurs euros.

* **On peut souhaiter qu'elle soit dérivable** pour éviter les *effets de tranche*: un centime de plus ou de moins ne doit
pas affecter significativement le taux marginal d'imposition (la pente de la courbe). C'est un point plus technique (et moins critique) que le précédent, mais il a un impact psychologique assez fort.

Bref vous l'aurez compris, cette *fonction fiscale* est pour moi un outil de débat sur la manière dont nous voulons vivre la solidarité et la redistribution. Pour ma part, **je trouverais intéressant qu'elle soit *concave* ** (c'est à dire que pour tout `x` et `y` on a: `ff(x+y) > ff(x) + ff(y)`).

Cela permet d'**introduire une forme de *subsidiarité* dans le système**: si un groupe de personne se met d'accord pour déclarer conjointement leurs revenus (et organiser localement le partage de la richesse), alors ils bénéficieront d'un *bonus fiscal*.

Cela aurait également un autre intérêt: celui d'inciter fortement les bas revenus et moins fortement les hauts revenus. 

On peut par exemple imaginer une fonction fiscale qui aurait la forme suivante:

In [6]:
pos = lambda x: 0.5*(x+np.abs(x))

import numpy as np

if False:
    smic=2343
    gmax = 6000
    x = np.arange(0,gmax)
    brut = x - 0.3742*x - pos(x-3086)*0.0808
    net = brut - 0.1370*x +pos(x-3086)*0.1045
    #y = (x>=1600*1.04)*(0.5*x) + (x<1600*1.04)*(514+0.62*x*0.5)
    #
    fig=plt.figure(figsize=(10,8))
    ax = plt.gca()
    #ax.plot(x,brut)
    #ax.plot(x,net)
    #ax.plot(x,500+0.35*x)
    impot = pos(net-808)*0.14 + pos(net-2232)*0.16 + pos(net-5985)*0.11
    rsa = (514 + 0.62*net - net)*(x<1.04*smic)
    dispo = np.sqrt(smic)*np.sqrt(x)
    ax.plot(x,dispo, 'g')
    #ax.plot(x,500+0.35*x, 'c')

    #ax.plot(x,y, 'r')
    ax.plot(smic+0*x, x, 'k:')
    ax.text(smic, -50, 'SMIC', ha='center', va='top')
    ax.set_ylim(0,gmax)
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['bottom'].set_color('none')
    plt.xticks([])
    plt.yticks([])

    plt.text(gmax*1.01, 0, 'capacité de production', ha='left', va='center')
    #plt.text(gmax*1.01, max(brut), 'salaire brut', color='b', ha='left', va='center')
    plt.text(gmax*1.01, max(x), 'revenu primaire', color='r', ha='left', va='center')
    #plt.text(gmax*1.01, max(net), 'salaire net', color='r', ha='left', va='center')
    #plt.text(gmax*1.02, max(dispo), 'revenu primaire', color='k', ha='left', va='center')
    plt.text(gmax*1.01, max(dispo), 'revenu disponible', color='g', ha='left', va='center')
    ax.set_aspect('equal')
    plt.plot(x,0*x,'k')
    plt.plot(0*x,x,'k')
    ax.plot(x,x, 'r-')
    ax.set_xlim(-100,8000)
    ax.set_ylim(-100,6000)
    pass
    fig.savefig('./img/2016-05-29-squareroot.png', bbox_inches='tight')

show_image('./img/2016-05-29-squareroot.png', width="600px")

Avec cet *impot positif-négatif progressif*, on taxe (fortement) les hauts revenus et on subventionne les revenus inférieurs au SMIC. L'incitation est d'autant plus forte que le revenu est faible, incitant chacun à:

* avoir une activité, même modeste
* déclarer les revenus de son activité

## Conclusion

J'avoue volontiers que ces réflexions ne contituent pas un projet clair. Pour être franc, je ne saurais même pas dire à quelle mouvance politique elles se rattachent. La combinaison du revenu universel et de la suppression du SMIC contitue pour moi un accord gagnant à tous les niveaux: c'est plus simple, c'est plus souple, les salariés sont protégés.

Mon but ici est surtout d'ouvrir le débat pour avoir votre opinion. Merci donc pour vos feedbacks et réactions.

In [7]:
HTML("""
<center>
<a href="https://twitter.com/share" class="twitter-share-button" data-via="Bibmartin_">Tweet</a>
<script>!function(d,s,id){var js,fjs=d.getElementsByTagName(s)[0],p=/^http:/.test(d.location)?'http':'https';if(!d.getElementById(id)){js=d.createElement(s);js.id=id;js.src=p+'://platform.twitter.com/widgets.js';fjs.parentNode.insertBefore(js,fjs);}}(document, 'script', 'twitter-wjs');</script>
</center>    
"""     
    +"<script>"
    "$('div.input').hide();"
    "$('div.output_prompt').css('display','none');"
    "$('div.input_prompt').css('display','none');"
    "</script>"
    )